In [29]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import csv

def pop():
    # 맨 위 뉴스 팝업
    first = browser.find_element(By.CSS_SELECTOR, '#news-results > div:nth-child(1) > div > div.cont > a > div > strong > span')
    time.sleep(2)
    first.click()

def gettitle():
    news_title = browser.find_element(By.CSS_SELECTOR, '#news-detail-modal > div > div > div.modal-body > div > div.news-view-head > h1.title')
    print('newstitle:', news_title.text)
    tmp["newstitle"].append(news_title.text)

def getbody():
    # 날짜 저장
    news_date = browser.find_elements(By.XPATH, '//*[@id="news-detail-modal"]/div/div/div[1]/div/div[1]/div[1]/ul/li[1]')
    for value in news_date:
        print('newsdate:',value.text)
        tmp["newsdate"].append(value.text)
    # 하나의 뉴스기사 기준 본문 저장
    news_bodies = browser.find_elements(By.XPATH, '//*[@id="news-detail-modal"]/div/div/div[1]/div/div[2]')
    news_body =''
    for i in news_bodies:
        news_body += i.text
    print('newsbody:',news_body)
    tmp["newsbody"].append(news_body)


# selenium
browser = webdriver.Chrome()

browser.set_window_size(1200,800)
url = "https://www.bigkinds.or.kr/v2/news/search.do"

browser.get(url)
time.sleep(0.2)

# 기간 설정
browser.find_element(By.XPATH, '//*[@id="collapse-step-1-body"]/div[3]/div/div[1]/div[1]/a').click()
browser.find_element(By.CSS_SELECTOR, '#search-begin-date').send_keys("2018-7-28")
time.sleep(2)

# 사건사고 분류 카테고리 - 범죄 선택
browser.find_element(By.XPATH, '//*[@id="collapse-step-2-body"]/div/div[1]/ul[1]/li[4]/a').click()
browser.find_element(By.XPATH,'//*[@id="filterTab04"]/li[1]/span').click()
time.sleep(2)

# 빈 데이터셋 생성
tmp = {"newsId":[],"newstitle":[],"newsdate":[],"newsbody":[]}


newsId = 0
iteration = 10**1

while (newsId < iteration):
    print('newsId:',newsId)
    tmp["newsId"].append(newsId)
    if (newsId % 10 == 0):
        time.sleep(2)
        pop()
    time.sleep(2)
    gettitle()
    getbody()
    if (newsId == 0):
        browser.find_element(By.CSS_SELECTOR,
            '#news-detail-modal > div > div > div.modal-body > div > div.list_prev_next > ul > li:nth-child(2) > dd > a > span').click()
    elif (newsId % 10 == 9 or newsId % 10 == 0 ):
        next_news = browser.find_element(By.CSS_SELECTOR, 
            '#news-detail-modal > div > div > div.modal-body > div > div.list_prev_next > ul > li:nth-child(3) > dd > a > span')
        time.sleep(2)
        next_news.click()
    else:
        next_news = browser.find_element(By.CSS_SELECTOR,
            '#news-detail-modal > div > div > div.modal-body > div > div.list_prev_next > ul > li.nextNewsItem > dd > a > span')
        time.sleep(2)
        next_news.click()
    newsId += 1
    
# save to csv
df = pd.DataFrame(tmp)
if not os.path.exists("dataset.csv"):
    df.to_csv("dataset.csv", index=False, mode="w", encoding="utf-8-sig")
else:
    df.to_csv('dataset.csv', index=False, mode="a", encoding="utf-8-sig", header=False)

print("-"*10,"Complete to acquisite.","-"*10)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=108.0.5359.71)
Stacktrace:
Backtrace:
	(No symbol) [0x00BFF243]
	(No symbol) [0x00B87FD1]
	(No symbol) [0x00A7CF10]
	(No symbol) [0x00AAD003]
	(No symbol) [0x00AA32A6]
	(No symbol) [0x00AC858C]
	(No symbol) [0x00AA2BFF]
	(No symbol) [0x00AC8804]
	(No symbol) [0x00ADC9EB]
	(No symbol) [0x00AC8386]
	(No symbol) [0x00AA163C]
	(No symbol) [0x00AA269D]
	GetHandleVerifier [0x00E99A22+2655074]
	GetHandleVerifier [0x00E8CA24+2601828]
	GetHandleVerifier [0x00CA8C0A+619850]
	GetHandleVerifier [0x00CA7830+614768]
	(No symbol) [0x00B905FC]
	(No symbol) [0x00B95968]
	(No symbol) [0x00B95A55]
	(No symbol) [0x00BA051B]
	BaseThreadInitThunk [0x75EC6939+25]
	RtlGetFullPathName_UEx [0x77B08FD2+1218]
	RtlGetFullPathName_UEx [0x77B08F9D+1165]
